In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import sys
sys.path.append('C:\\Users\\Graúna Motos\\Documents\\credit_risk')
#sys.path.append('/home/noise/Documentos/credit_risk')
from src.utils_modelling import analisy_univariate, wo_discretize, plot_woe, wo_discretize_continuos
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from src.utils_modelling import LogisticRegressionPvalues
import mlflow
import warnings

warnings.filterwarnings('ignore')

In [2]:

'''#LINUX
input_train = pd.read_csv('/home/noise/Documentos/credit_risk/data/loan_data_inputs_train.csv', index_col=0)
target_train = pd.read_csv('/home/noise/Documentos/credit_risk/data/loan_data_targets_train.csv', index_col=0, header= None)
input_test = pd.read_csv('/home/noise/Documentos/credit_risk/data/loan_data_inputs_test.csv', index_col=0)
target_test = pd.read_csv('/home/noise/Documentos/credit_risk/data/loan_data_targets_test.csv', index_col=0, header= None)
'''

"#LINUX\ninput_train = pd.read_csv('/home/noise/Documentos/credit_risk/data/loan_data_inputs_train.csv', index_col=0)\ntarget_train = pd.read_csv('/home/noise/Documentos/credit_risk/data/loan_data_targets_train.csv', index_col=0, header= None)\ninput_test = pd.read_csv('/home/noise/Documentos/credit_risk/data/loan_data_inputs_test.csv', index_col=0)\ntarget_test = pd.read_csv('/home/noise/Documentos/credit_risk/data/loan_data_targets_test.csv', index_col=0, header= None)\n"

In [3]:
input_train = pd.read_csv('C:\\Users\\Graúna Motos\\Documents\\credit_risk\\data\\silver\\loan_data_inputs_train.csv', index_col=0)
target_train = pd.read_csv('C:\\Users\\Graúna Motos\\Documents\\credit_risk\\data\\silver\\loan_data_targets_train.csv', index_col=0, header=None  )
input_test = pd.read_csv('C:\\Users\\Graúna Motos\\Documents\\credit_risk\\data\\silver\\loan_data_inputs_test.csv', index_col=0)
target_test = pd.read_csv('C:\\Users\\Graúna Motos\\Documents\\credit_risk\\data\\silver\\loan_data_targets_test.csv', index_col=0, header=None)

In [4]:
target_train.head()

,1
0,
NaN,good_bad
427211.0,1
206088.0,1
136020.0,1
412305.0,0


In [5]:
# Remove the 'good_bad' row and convert to numeric
target_train = target_train.apply(pd.to_numeric, errors='coerce')

# Remove any remaining non-numeric values
target_train = target_train.dropna()

# Reset the index
target_train = target_train.reset_index(drop=True)



In [6]:
target_train.shape


(373028, 1)

In [7]:
target_train.shape


(373028, 1)

# Train model PD

In [8]:
inputs_train_with_ref_cat = input_train.loc[: , ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ_smb_wedd_ren_en_mov_house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth_med_vacation',
'purpose:major_car_home',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',
'delinq_2yrs:0',
'delinq_2yrs:1-3',
'delinq_2yrs:>=4',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'open_acc:0',
'open_acc:1-3',
'open_acc:4-12',
'open_acc:13-17',
'open_acc:18-22',
'open_acc:23-25',
'open_acc:26-30',
'open_acc:>=31',
'pub_rec:0-2',
'pub_rec:3-4',
'pub_rec:>=5',
'total_acc:<=27',
'total_acc:28-51',
'total_acc:>=52',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'total_rev_hi_lim:<=5K',
'total_rev_hi_lim:5K-10K',
'total_rev_hi_lim:10K-20K',
'total_rev_hi_lim:20K-30K',
'total_rev_hi_lim:30K-40K',
'total_rev_hi_lim:40K-55K',
'total_rev_hi_lim:55K-95K',
'total_rev_hi_lim:>95K',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>86',
]]    

In [9]:
ref_categories = [
    'grade:G',
    'home_ownership:RENT_OTHER_NONE_ANY',
    'addr_state:ND_NE_IA_NV_FL_HI_AL',
    'verification_status:Verified',
    'purpose:educ_smb_wedd_ren_en_mov_house',  
    'initial_list_status:f',
    'term:60',
    'emp_length:0',
    'mths_since_issue_d:>84',
    'int_rate:>20.281',
    'mths_since_earliest_cr_line:<140',
    'delinq_2yrs:>=4',
    'inq_last_6mths:>6',
    'open_acc:0',
    'pub_rec:0-2',
    'total_acc:<=27',
    'acc_now_delinq:0',
    'total_rev_hi_lim:<=5K',
    'annual_inc:<20K',
    'dti:>35',
    'mths_since_last_delinq:0-3',
    'mths_since_last_record:0-2'
]

In [10]:
input_train = inputs_train_with_ref_cat.drop(ref_categories, axis=1)
input_train.head()

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_VA,addr_state:NY,...,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
427211,True,False,False,False,False,False,False,True,0,False,...,1,0,0,0,1,0,0,0,0,0
206088,False,False,True,False,False,False,False,True,0,False,...,0,1,0,0,1,0,0,0,0,0
136020,True,False,False,False,False,False,False,True,0,False,...,0,0,1,0,1,0,0,0,0,0
412305,False,False,False,True,False,False,False,False,0,False,...,0,1,0,0,1,0,0,0,0,0
36159,False,False,True,False,False,False,False,True,0,False,...,1,0,0,0,1,0,0,0,0,0


# Logistic Regression Model

In [11]:
reg = LogisticRegression()

In [12]:
input_train.shape
#target_train.shape

(373028, 104)

In [13]:
print(target_train)

          1
0       1.0
1       1.0
2       1.0
3       0.0
4       0.0
...     ...
373023  1.0
373024  1.0
373025  1.0
373026  1.0
373027  1.0

[373028 rows x 1 columns]


In [14]:
target_train.shape

(373028, 1)

In [15]:
target_train.head()

,1
0,1.0
1,1.0
2,1.0
3,0.0
4,0.0


In [16]:
input_train.head()

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_VA,addr_state:NY,...,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
427211,True,False,False,False,False,False,False,True,0,False,...,1,0,0,0,1,0,0,0,0,0
206088,False,False,True,False,False,False,False,True,0,False,...,0,1,0,0,1,0,0,0,0,0
136020,True,False,False,False,False,False,False,True,0,False,...,0,0,1,0,1,0,0,0,0,0
412305,False,False,False,True,False,False,False,False,0,False,...,0,1,0,0,1,0,0,0,0,0
36159,False,False,True,False,False,False,False,True,0,False,...,1,0,0,0,1,0,0,0,0,0


In [17]:
import numpy as np
from scipy import stats

In [18]:
from sklearn import linear_model
import numpy as np
import scipy.stats as stat
import statsmodels.api as sm
import pandas as pd

class LogisticRegressionP:
    def __init__(self, alpha=0.0001, method='l1'):
        self.alpha = alpha
        self.method = method
        self.model = None
        self.removed_cols = []
        
    def _check_data(self, X):
        """
        Verifica e processa os dados de entrada
        """
        X = X.copy()
        
        # Converte todas as colunas para numéricas, tratando erros
        for col in X.columns:
            try:
                X[col] = pd.to_numeric(X[col], errors='raise')
            except:
                print(f"Warning: Column {col} could not be converted to numeric. Removing it.")
                X = X.drop(columns=[col])
                self.removed_cols.append(col)
        
        # Verifica variância zero
        constant_cols = []
        for col in X.columns:
            if X[col].std() < 1e-6:
                constant_cols.append(col)
                
        if constant_cols:
            print(f"Warning: Columns with near-zero variance: {constant_cols}")
            X = X.drop(columns=constant_cols)
            self.removed_cols.extend(constant_cols)
            
        # Verifica multicolinearidade
        if X.shape[1] > 1:
            corr_matrix = X.corr().abs()
            upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
            high_corr = [(i, j) for i, j in zip(*np.where(upper > 0.95))]
            
            if high_corr:
                print(f"Warning: High correlation (>0.95) between features: {high_corr}")
                # Opcional: remover uma das colunas altamente correlacionadas
                cols_to_remove = [pair[1] for pair in high_corr]
                if cols_to_remove:
                    X = X.drop(columns=X.columns[cols_to_remove])
                    self.removed_cols.extend(X.columns[cols_to_remove])
                    print(f"Removed highly correlated columns: {X.columns[cols_to_remove]}")
        
        # Verifica se há valores nulos
        if X.isnull().any().any():
            print("Warning: Data contains null values. Filling with 0...")
            X = X.fillna(0)
            
        return X
    
    def fit(self, X, y):
        try:
            # Converte para DataFrame se não for
            if not isinstance(X, pd.DataFrame):
                X = pd.DataFrame(X)
            
            # Converte y para numpy array se não for
            if isinstance(y, pd.Series):
                y = y.values
            elif isinstance(y, pd.DataFrame):
                y = y.values.ravel()
            elif not isinstance(y, np.ndarray):
                y = np.array(y)
            
            # Converte y para float64
            y = y.astype(np.float64)
                
            # Garante que y seja um array 2D
            if len(y.shape) == 1:
                y = y.reshape(-1, 1)
                
            # Checa e limpa os dados
            X = self._check_data(X)
            
            # Verifica se ainda há colunas após o processamento
            if X.shape[1] == 0:
                raise ValueError("No columns remained after data processing!")
            
            # Converte para array numpy
            X_array = X.astype(np.float64)
            
            # Adiciona constante
            X_copy = sm.add_constant(X_array)
            
            # Tenta fit com diferentes valores de alpha se necessário
            try:
                self.model = sm.Logit(y, X_copy).fit_regularized(
                    alpha=self.alpha, 
                    method=self.method,
                    maxiter=1000,
                    cnvrg_tol=1e-8
                )
            except np.linalg.LinAlgError:
                print("Warning: Singular matrix detected. Trying with higher regularization...")
                self.alpha = max(0.01, self.alpha * 10)
                self.model = sm.Logit(y, X_copy).fit_regularized(
                    alpha=self.alpha,
                    method=self.method
                )
                
        except Exception as e:
            print(f"Error during fitting: {str(e)}")
            print("\nShape of X:", X.shape)
            print("Shape of y:", y.shape)
            print("X dtypes:\n", X.dtypes)
            print("y dtype:", y.dtype)
            raise e

    def predict(self, X):
        try:
            if not isinstance(X, pd.DataFrame):
                X = pd.DataFrame(X)
            
            # Remove as colunas que foram removidas durante o fit
            if self.removed_cols:
                X = X.drop(columns=list(set(self.removed_cols) & set(X.columns)))
                
            X = self._check_data(X)
            X_array = X.astype(np.float64)
            X_copy = sm.add_constant(X_array)
            
            return self.model.predict(X_copy)
            
        except Exception as e:
            print(f"Error during prediction: {str(e)}")
            raise e
    
    def get_summary_table(self):
        try:
            summary = self.model.summary2().tables[1]
            summary['Wald'] = summary['z'] ** 2
            summary_table = summary[['Coef.', 'P>|z|', 'Wald']]
            summary_table.columns = ['Beta Coefficient', 'P-Value', 'Wald Statistic']
            summary_table = summary_table.sort_index()
            summary_table['P-Value'] = summary_table['P-Value'].apply(lambda x: '{:.20f}'.format(x))   
            return summary_table
            
        except Exception as e:
            print(f"Error getting summary table: {str(e)}")
            raise e

In [19]:
feature_name = input_train.columns.values
# Stores the names of the columns of a dataframe in a variable.

In [20]:
target_train.head()

,1
0,1.0
1,1.0
2,1.0
3,0.0
4,0.0


In [21]:
target_train

,1
0,1.0
1,1.0
2,1.0
3,0.0
4,0.0
...,...
373023,1.0
373024,1.0
373025,1.0
373026,1.0


In [22]:
'''reg2 = LogisticRegressionP(alpha=1, method='l1')
reg2.fit(input_train, target_train)'''


"reg2 = LogisticRegressionP(alpha=1, method='l1')\nreg2.fit(input_train, target_train)"

In [23]:
'''summary_table = reg2.get_summary_table()'''

'summary_table = reg2.get_summary_table()'

In [24]:
'''summary_table.head(200)'''

'summary_table.head(200)'

In [25]:
# Define columns to drop
columns_to_drop = [
    'open_acc:1-3', 'open_acc:13-17', 'open_acc:18-22', 'open_acc:23-25',
    'open_acc:26-30', 'open_acc:4-12', 'open_acc:>=31', 'pub_rec:3-4', 'pub_rec:>=5',
    'term:36', 'total_acc:28-51', 'total_acc:>=52',
    'emp_length:1', 'emp_length:10', 'emp_length:2-4', 'emp_length:5-6', 'emp_length:7-9',
    'mths_since_earliest_cr_line:>352'
]

# Drop the columns from input_train
input_train = input_train.drop(columns=columns_to_drop)



In [40]:
mlflow.set_tracking_uri(uri='http://localhost:5001')
mlflow.set_experiment(experiment_id=1)
mlflow.autolog()

2024/11/01 11:28:06 WARNING mlflow.utils.autologging_utils: MLflow statsmodels autologging is known to be compatible with 0.11.1 <= statsmodels <= 0.14.3, but the installed version is 0.14.4. If you encounter errors during autologging, try upgrading / downgrading statsmodels to a compatible version, or try upgrading MLflow.
2024/11/01 11:28:06 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2024/11/01 11:28:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [42]:
import os
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression

# Configuração do URI de tracking do MLflow e das variáveis de ambiente para o MinIO
mlflow.set_tracking_uri("http://localhost:5001")  # URI do MLflow ajustado para a porta 5001
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9005"  # Porta do MinIO ajustada para 9005
os.environ["AWS_ACCESS_KEY_ID"] = "mlflow"
os.environ["AWS_SECRET_ACCESS_KEY"] = "password"

# Dados de entrada para o treinamento do modelo (defina input_train e target_train antes do uso)
# Exemplo de definição (substitua com os dados reais):
# input_train, target_train = <seu_código_para_carregar_dados>()

# Treinamento do modelo e logging no MLflow
try:
    with mlflow.start_run():
        # Treinamento do modelo
        reg3 = LogisticRegression()
        reg3.fit(input_train, target_train)  # Defina input_train e target_train antes do uso

        # Log de parâmetros do modelo
        mlflow.log_param("model_type", "LogisticRegression")
        
        # Log de métricas
        train_score = reg3.score(input_train, target_train)
        mlflow.log_metric("train_accuracy", train_score)
        
        # Log do modelo no MLflow
        mlflow.sklearn.log_model(reg3, "model")
        
        print("Execução e log concluídos com sucesso.")
except Exception as e:
    print(f"Ocorreu um erro ao iniciar a execução do MLflow: {e}")


2024/11/01 11:30:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/01 11:30:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run clean-ram-92 at: http://localhost:5001/#/experiments/1/runs/a9054243c78742d6981d65417a8d5123.
2024/11/01 11:30:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/1.


Execução e log concluídos com sucesso.


In [28]:

# Desbloqueando as opções de exibição do pandas
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.width = None
